# Conor's CNN + CSV Ensemble attempt

# 1 - Bring the image to the dataframe

In [4]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Input, Conv1D, MaxPooling1D, Flatten, Conv2D, MaxPooling2D, Normalization, GlobalAveragePooling2D

# Define constants
GENRES = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
FILE_PATH = os.path.join('Data', 'mel_spectrograms', 'mel_spectrogram_512')
CSV_PATH = "Data/features_30_sec.csv"

# Extract song file paths
song_to_filepaths = {}
for genre in GENRES:
    genre_dir = os.path.join(FILE_PATH, genre)
    for file in os.listdir(genre_dir):
        if not file.endswith(".png"):
            continue
        song_id = file.split("_clip_")[0]  # Extract song ID
        if song_id not in song_to_filepaths:
            song_to_filepaths[song_id] = []
        song_to_filepaths[song_id].append(os.path.join(genre_dir, file))

# Load dataset
data = pd.read_csv(CSV_PATH)
df = data.drop(columns=['length'])  # Remove 'length' column
df.rename(columns={'label': 'genre'}, inplace=True)

# Add filepath column
df['filepath'] = df.apply(lambda row: os.path.join(FILE_PATH, row['genre'], row['filename'].replace('.wav', '') + '.png'), axis=1)

# Encode genres
label_encoder = LabelEncoder()
df['genre_encoded'] = label_encoder.fit_transform(df['genre'])

X = df.drop(columns=['filename'])
y = df['genre_encoded']

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_filepaths = X_train['filepath']
y_train_filepaths_genres = X_train['genre']
X_test_filepaths = X_test['filepath']
y_test_filepaths_genres = X_test['genre']

X_train = X_train.drop(columns=['filepath', 'genre'])
X_test = X_test.drop(columns=['filepath', 'genre'])

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_test_scaled.shape, y_test.shape, X_train_scaled.shape, y_train.shape
print(y_test_filepaths_genres[0:10])
print(y_test[0:10])


521         jazz
737          pop
740          pop
660        metal
411       hiphop
678        metal
626        metal
513         jazz
859       reggae
136    classical
Name: genre, dtype: object
521    5
737    7
740    7
660    6
411    4
678    6
626    6
513    5
859    8
136    1
Name: genre_encoded, dtype: int64


## 2 - SVC

In [5]:
# Train and evaluate SVM model
svm_model = SVC(probability=True)
svm_model.fit(X_train_scaled, y_train)
y_pred = svm_model.predict(X_test_scaled)
svc_accuracy = accuracy_score(y_test, y_pred)
print(f"SVM accuracy: {svc_accuracy}")


SVM accuracy: 0.86


## 3 - XGBoost

In [6]:
# Make an XGBoost model
from xgboost import XGBClassifier

xgb_model = XGBClassifier()
xgb_model.fit(X_train_scaled, y_train)
y_pred = xgb_model.predict(X_test_scaled)
xgboost_accuracy = accuracy_score(y_test, y_pred)
print(f"XGBoost accuracy: {xgboost_accuracy}")

XGBoost accuracy: 1.0


## 4 - Random Forest Classifier

In [7]:
## Make a Random Forest model
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()
rf_model.fit(X_train_scaled, y_train)
y_pred = rf_model.predict(X_test_scaled)
rfc_accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest accuracy: {rfc_accuracy}")

Random Forest accuracy: 0.93


## 5 - Dense CNN on .csv data only

In [8]:
# Define the Dense Neural Network (MLP) architecture

dnn_model = Sequential([
    Input(shape=(58, )),
    
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(64, activation='relu'),
    BatchNormalization(),
    
    Dense(len(label_encoder.classes_), activation='softmax')
])

dnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
dnn_model.summary()

dnn_model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test), epochs=100, batch_size=32)

# Evaluate the model
_, dense_cnn_accuracy = dnn_model.evaluate(X_test_scaled, y_test)
print(f"DNN accuracy: {dense_cnn_accuracy}")



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │        30,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 207,178 (809.29 KB)

 Trainable params: 205,258 (801.79 KB)

 Non-trainable params: 1,920 (7.50 KB)

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.1804 - loss: 2.5345 - val_accuracy: 0.3350 - val_loss: 1.9901
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4472 - loss: 1.5986 - val_accuracy: 0.5500 - val_loss: 1.6868
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5669 - loss: 1.2809 - val_accuracy: 0.6450 - val_loss: 1.4368
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6501 - loss: 1.0270 - val_accuracy: 0.7000 - val_loss: 1.1835
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7035 - loss: 0.8571 - val_accuracy: 0.7500 - val_loss: 0.9736
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7234 - loss: 0.7787 - val_accuracy: 0.7850 - val_loss: 0.8162
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7872 - loss: 0.6781 - val_accuracy: 0.7900 - val_loss: 0.7194
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7766 - loss: 0.6540 - val_accuracy: 0.8150 - v

## 6 - Conv1D CNN on .csv data only

In [9]:
# Define the CNN model architecture
cnn_csv_model = Sequential([
    # First convolutional layer with 'same' padding
    Input(shape = (58, 1)),
    Conv1D(64, 3, activation='relu', padding='same',),
    MaxPooling1D(2, padding='same'),  # MaxPooling2D with same padding

    # Second convolutional layer with 'same' padding
    Conv1D(128, 3, activation='relu', padding='same'),
    MaxPooling1D(2, padding='same'),

    # Third convolutional layer with 'same' padding
    Conv1D(256, 3, activation='relu', padding='same'),
    MaxPooling1D(2, padding='same'),

    # Flatten the output of the convolutional layers
    Flatten(),

    # Fully connected layers
    Dense(512, activation='relu'),
    Dropout(0.5),

    # Output layer with softmax activation
    Dense(10, activation='softmax')
])

# Compile the model
cnn_csv_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
cnn_csv_model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test), epochs=100, batch_size=32)

# Evaluate the model
_, conv1d_cnn_accuracy = cnn_csv_model.evaluate(X_test_scaled, y_test)
print(f"CNN accuracy: {conv1d_cnn_accuracy}")

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.2116 - loss: 2.1426 - val_accuracy: 0.3700 - val_loss: 1.6898
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4186 - loss: 1.5282 - val_accuracy: 0.5500 - val_loss: 1.3190
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6377 - loss: 1.0530 - val_accuracy: 0.7050 - val_loss: 0.8396
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7678 - loss: 0.6877 - val_accuracy: 0.7300 - val_loss: 0.7328
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8533 - loss: 0.4379 - val_accuracy: 0.7650 - val_loss: 0.6060
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8861 - loss: 0.3193 - val_accuracy: 0.8500 - val_loss: 0.4528
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9308 - loss: 0.2322 - val_accuracy: 0.8300 - val_loss: 0.4827
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9213 - loss: 0.2261 - val_accuracy: 0.8450 - 

## 7 - Image CNN

In [10]:

# Load in a downloaded model

from tensorflow.keras.models import load_model
import numpy as np
import tensorflow as tf

model = load_model('music_genre_classification_model.h5')

# Augmentation function
def augment_image(image):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    return image

X_train_images = []
X_test_images = []

for filepath in X_train_filepaths:
    img = tf.io.read_file(filepath)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.convert_image_dtype(img, dtype=tf.float32)
    img = tf.image.resize(img, [256, 256])  # Resize to 256x256
    img = tf.image.rgb_to_grayscale(img)  # Convert to grayscale
    img = augment_image(img)
    img = img.numpy()
    X_train_images.append(img)

for filepath in X_test_filepaths:
    img = tf.io.read_file(filepath)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.convert_image_dtype(img, dtype=tf.float32)
    img = tf.image.resize(img, [256, 256])  # Resize to 256x256
    img = tf.image.rgb_to_grayscale(img)  # Convert to grayscale
    img = augment_image(img)
    img = img.numpy()
    X_test_images.append(img)

# Convert to numpy arrays
X_train_images = np.array(X_train_images)
X_train_images = X_train_images.reshape(X_train_images.shape[0], 256, 256, 1)  # Shape should match (None, 256, 256, 1)
X_test_images = np.array(X_test_images)
X_test_images = X_test_images.reshape(X_test_images.shape[0], 256, 256, 1)  # Shape should match (None, 256, 256, 1)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

_, accuracy = model.evaluate(X_test_images, y_test)  # Use X_test_images here
print(f"Loaded model accuracy: {accuracy}")


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - accuracy: 0.1843 - loss: 2.2376
Loaded model accuracy: 0.1550000011920929


## 8 - Soft Voting Ensemble

In [11]:
import numpy as np

svc_predictions = svm_model.predict_proba(X_test_scaled)
xgb_predictions = xgb_model.predict_proba(X_test_scaled)
rfc_predictions = rf_model.predict_proba(X_test_scaled)
dnn_predictions = dnn_model.predict(X_test_scaled)
cnn_predictions = cnn_csv_model.predict(X_test_scaled)
cnn2_predictions = model.predict(X_test_images)

# Compute the average accuracy across the models
average_prediction_proba = (svc_predictions + xgb_predictions + rfc_predictions + dnn_predictions + cnn_predictions + cnn2_predictions) / 6
best = np.argmax(average_prediction_proba, axis=1)
average_accuracies = accuracy_score(y_test, best)

# Print out the individual accuracies and the average accuracy
print(f"SVC Accuracy: {svc_accuracy}")
print(f"XGBoost Accuracy: {xgboost_accuracy}")
print(f"Random Forest Accuracy: {rfc_accuracy}")
print(f"Dense CNN Accuracy: {dense_cnn_accuracy}")
print(f"Conv1D CNN Accuracy: {conv1d_cnn_accuracy}")
print(f"Average Accuracy: {average_accuracies}")
print(f"Loaded Model Accuracy: {accuracy}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 140ms/step
SVC Accuracy: 0.86
XGBoost Accuracy: 1.0
Random Forest Accuracy: 0.93
Dense CNN Accuracy: 0.8949999809265137
Conv1D CNN Accuracy: 0.8899999856948853
Average Accuracy: 0.935
Loaded Model Accuracy: 0.1550000011920929


## 9 - Hard Voting Ensemble

In [12]:
from scipy.stats import mode

svc_predictions = svm_model.predict(X_test_scaled)
xgb_predictions = xgb_model.predict(X_test_scaled)
rfc_predictions = rf_model.predict(X_test_scaled)
dnn_predictions = np.argmax(dnn_model.predict(X_test_scaled), axis=1)
cnn_predictions = np.argmax(cnn_csv_model.predict(X_test_scaled), axis=1)
cnn2_predictions = np.argmax(model.predict(X_test_images), axis=1)

# Combine the predictions
combined_predictions = np.array([svc_predictions, xgb_predictions, rfc_predictions, dnn_predictions, cnn_predictions, cnn2_predictions])

# Take the mode of the predictions
combined_predictions, _ = mode(combined_predictions, axis=0, keepdims=True)
combined_predictions = combined_predictions.flatten()

# Calculate the accuracy
accuracy = accuracy_score(y_test, combined_predictions)
print(f"Combined Model Accuracy: {accuracy}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 142ms/step
Combined Model Accuracy: 0.95


## 10 - Weighting Voting

In [13]:
import numpy as np
from sklearn.metrics import accuracy_score

accuracy_svm = accuracy_score(y_test, svm_model.predict(X_test_scaled))
xgb_accuracy = accuracy_score(y_test, xgb_model.predict(X_test_scaled))
accuracy_rf = accuracy_score(y_test, rf_model.predict(X_test_scaled))
dnn_accuracy = accuracy_score(y_test, np.argmax(dnn_model.predict(X_test_scaled), axis=1))
cnn_csv_accuracy = accuracy_score(y_test, np.argmax(cnn_csv_model.predict(X_test_scaled), axis=1))
cnn2_accuracy = accuracy_score(y_test, np.argmax(model.predict(X_test_images), axis=1))

# Define the weights based on model accuracies (ensure these variables are defined with the correct accuracies)
weights = {
    'svm': accuracy_svm,
    'xgb': xgb_accuracy,
    'rf': accuracy_rf,
    'dnn': dnn_accuracy,  # Make sure to replace with the actual accuracy of dnn_model
    'cnn_csv': cnn_csv_accuracy,  # Make sure to replace with the actual accuracy of cnn_csv_model
    'cnn2': cnn2_accuracy
}

# Normalize the weights so they sum to 1
total_weight = sum(weights.values())
weights = {k: v / total_weight for k, v in weights.items()}

# Get predictions from each model
xgb_preds_proba = xgb_model.predict_proba(X_test_scaled)
rf_preds_proba = rf_model.predict_proba(X_test_scaled)
svm_preds_proba = svm_model.predict_proba(X_test_scaled)
dnn_preds_proba = dnn_model.predict(X_test_scaled)  # Assuming dnn_model has predict_proba()
cnn_csv_preds_proba = cnn_csv_model.predict(X_test_scaled)  # Assuming cnn_csv_model has predict_proba()
cnn2_preds_proba = model.predict(X_test_images)  # Assuming model has predict_proba()

# Apply weights to the predicted probabilities
weighted_xgb_proba = xgb_preds_proba * weights['xgb']
weighted_rf_proba = rf_preds_proba * weights['rf']
weighted_svm_proba = svm_preds_proba * weights['svm']
weighted_dnn_proba = dnn_preds_proba * weights['dnn']
weighted_cnn_csv_proba = cnn_csv_preds_proba * weights['cnn_csv']
weighted_cnn2_proba = cnn2_preds_proba * weights['cnn2']

# Combine the weighted probabilities
weighted_avg_proba = weighted_xgb_proba + weighted_rf_proba + weighted_svm_proba + weighted_dnn_proba + weighted_cnn_csv_proba

# Convert weighted probabilities to class predictions
weighted_ensemble_preds = np.argmax(weighted_avg_proba, axis=1)

# Evaluate the weighted ensemble performance
weighted_ensemble_accuracy = accuracy_score(y_test, weighted_ensemble_preds)
print(f"Weighted Ensemble Accuracy: {weighted_ensemble_accuracy:.3f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 136ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step
Weighted Ensemble Accuracy: 0.935


## 11 - Stacked Ensemble

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Step 1: Get predictions from base models
xgb_train_preds = xgb_model.predict_proba(X_train_scaled)
rf_train_preds = rf_model.predict_proba(X_train_scaled)
svm_train_preds = svm_model.predict_proba(X_train_scaled)
dnn_train_predictions = dnn_model.predict(X_train_scaled)
cnn_train_preds = cnn_csv_model.predict(X_train_scaled)
cnn2_train_predictions = model.predict(X_train_images)

xgb_test_preds = xgb_model.predict_proba(X_test_scaled)
rf_test_preds = rf_model.predict_proba(X_test_scaled)
svm_test_preds = svm_model.predict_proba(X_test_scaled)
dnn_test_predictions = dnn_model.predict(X_test_scaled)
cnn_test_preds = cnn_csv_model.predict(X_test_scaled)
cnn2_test_predictions = model.predict(X_test_images)

# Step 2: Create new dataset for meta-model training
stacked_train = np.hstack((xgb_train_preds, svm_train_preds, cnn_train_preds,  rf_train_preds + dnn_train_predictions + cnn_train_preds + cnn2_train_predictions))
stacked_test = np.hstack((xgb_test_preds, svm_test_preds, cnn_test_preds, rf_test_preds + dnn_test_predictions + cnn_test_preds + cnn2_test_predictions))

# Step 3: Train meta-model
meta_model = LogisticRegression()
meta_model.fit(stacked_train, y_train)

# Step 4: Predict with the meta-model
ensemble_preds = meta_model.predict(stacked_test)

# Step 5: Evaluate performance
stack_ensemble_accuracy = accuracy_score(y_test, ensemble_preds)
print(f"Stacking Ensemble Accuracy: {stack_ensemble_accuracy:.3f}")

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 161ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 146ms/step
Stacking Ensemble Accuracy: 0.920


## Attempted combination CNN

In [ ]:
from tensorflow.keras.layers import concatenate

final_cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)),
    Normalization(),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    Normalization(),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    Normalization(),
    MaxPooling2D((2, 2)),
    
    Conv2D(256, (3, 3), activation='relu'),
    Normalization(),
    MaxPooling2D((2, 2)),
    
    Flatten()
    ])

final_csv_model = Sequential([
    Input(shape = (58, 1)),
    Conv1D(64, 3, activation='relu', padding='same',),
    MaxPooling1D(2, padding='same'),  # MaxPooling2D with same padding

    # Second convolutional layer with 'same' padding
    Conv1D(128, 3, activation='relu', padding='same'),
    MaxPooling1D(2, padding='same'),

    # Third convolutional layer with 'same' padding
    Conv1D(256, 3, activation='relu', padding='same'),
    MaxPooling1D(2, padding='same'),

    # Flatten the output of the convolutional layers
    Flatten()
])

image_input = Input(shape=(256, 256, 1))
csv_input = Input(shape=(58, 1))

image_features = final_cnn_model(image_input)
csv_features = final_csv_model(csv_input)

merged = concatenate([image_features, csv_features])
output = Dense(512, activation='relu')(merged)
output = BatchNormalization()(output)
output = Dropout(0.5)(output)

output = Dense(256, activation='relu')(output)
output = BatchNormalization()(output)
output = Dropout(0.4)(output)

output = Dense(128, activation='relu')(output)
output = Dense(10, activation='softmax')(output)

# Compile the final model
final_model = tf.keras.Model(inputs=[image_input, csv_input], outputs=output)

final_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the final model
final_model.fit([X_train_images, X_train_scaled], y_train, validation_data=([X_test_images, X_test_scaled], y_test), epochs=20, batch_size=32)

# Evaluate the final model
final_accuracy = final_model.evaluate([X_test_images, X_test_scaled], y_test)
print(f"Final Model Accuracy: {final_accuracy[1]:.3f}")

/Users/conorwoollatt/.pyenv/versions/3.9.6/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 17s 640ms/step - accuracy: 0.2345 - loss: 2.4131 - val_accuracy: 0.1000 - val_loss: 2.3678
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 15s 613ms/step - accuracy: 0.4368 - loss: 1.6067 - val_accuracy: 0.1700 - val_loss: 2.0501
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 16s 624ms/step - accuracy: 0.6188 - loss: 1.1231 - val_accuracy: 0.4300 - val_loss: 1.7981
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 16s 626ms/step - accuracy: 0.7818 - loss: 0.6497 - val_accuracy: 0.3800 - val_loss: 1.6234
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 16s 628ms/step - accuracy: 0.8767 - loss: 0.4095 - val_accuracy: 0.2600 - val_loss: 1.9201
Epoch 6/20
 6/25 ━━━━━━━━━━━━━━━━━━━━ 10s 572ms/step - accuracy: 0.8918 - loss: 0.3352

KeyboardInterrupt: 